In [72]:
import os
import pickle

import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn import svm, tree, linear_model
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

# 장르 추가로 정확도 높이기

영화와 영화에 대한 사용자의 취향을 학습한 기존 추천 모델에 장르와 장르에 대한 사용자의 취향을 학습한 장르 추천 모델 데이터를 더해 영화 추천 정확도를 높일 수 있는지 확인합니다.

각 벡터를 모두 이어 64 차원의 벡터를 input으로, prefer 여부인 1, 0을 output으로 하는 모델을 학습합니다.

`장르 8 + 영화 24 + 장르 사용자 8 + 영화 사용자 24 = 64`

장르게 여러 개인 영화의 경우 장르 벡터들의 평균을 사용했습니다.

SVM, Decision Tree, Logistic Regression 모델을 시도했습니다.

이 중 SVM만이 79.73%로 기존 77.04%보다 더 높은 정확도를 보였습니다.

## 학습 데이터 만들기

학습 데이터는 사용자의 별점, 영화의 장르, 각 모델의 벡터로 이뤄집니다.

먼저, 사용자의 별점, 영화의 장르 데이터를 로드하고, 별점별 영화와 장르를 알 수 있게 join합니다.

In [2]:
rating_df = pd.read_csv('./The Movies Dataset/processed_ratings.csv')
movie_df = pd.read_csv('./The Movies Dataset/processed_movies.csv')
genre_df = pd.read_csv('./The Movies Dataset/genres.csv')

In [3]:
movie_df.query('genres.isna()')

,title,genres,original_language,release_date,imdbId
55,Kids of the Round Table,NaN,en,1997-07-08,113541
83,Last Summer in the Hamptons,NaN,en,1995-11-22,113612
126,Jupiter's Wife,NaN,en,1995-01-01,110217
137,Target,NaN,en,1995-08-01,114618
390,Desert Winds,NaN,en,1997-12-31,112849
...,...,...,...,...,...
45424,The Untameable Whiskers,NaN,fr,1904-03-05,135631
45425,The Imperceptable Transmutations,NaN,fr,1904-01-01,224286
45432,St. Michael Had a Rooster,NaN,it,1972-01-01,69215
45441,Satan Triumphant,NaN,en,1917-10-21,8536


In [4]:
genre_rating_df = (
    rating_df
    .assign(
        prefer=lambda df: (df['rating'] >= 3.5).astype(int),
    )
    .join(
        movie_df
        .query('~genres.isna()')
        [['imdbId', 'genres']]
        .set_index('imdbId'), 
        on='imdbId', 
        how='inner',
    )
)
genre_rating_df

,userId,rating,timestamp,imdbId,prefer,genres
0,1,1.0,1425941529,112573,0,"Action, Drama, History, War"
1,11,3.5,1231676989,112573,1,"Action, Drama, History, War"
2,22,5.0,1111937009,112573,1,"Action, Drama, History, War"
3,24,5.0,979870012,112573,1,"Action, Drama, History, War"
4,29,3.0,1044020005,112573,0,"Action, Drama, History, War"
...,...,...,...,...,...,...
26024283,270887,4.5,1487299002,469810,1,"Thriller, Drama"
26024285,270887,5.0,1486961830,487156,1,"Action, Thriller"
26024286,270887,4.0,1479088587,3731196,1,"Thriller, Drama"
26024287,270887,4.0,1493084042,176326,1,"Drama, Thriller"


장르 추천 모델과 영화 추천 모델을 불러옵니다.

`*2idx` 정보를 불러와 사용자, 영화, 장르의 ID와 모델의 벡터 위치를 매핑합니다.

테스트 데이터셋도 함께 가져와 기준이 되는 모델 정확도를 다시 한 번 확인합니다.

이 경우 baseline은 기존 영화 추천 모델의 정확도인 `0.7704`입니다.

In [5]:
def load_dataset(filepath):
    with np.load(filepath) as filedata:
        return dict(filedata['uid2idx']), dict(filedata['iid2idx']), \
            filedata['train_x'], filedata['train_y'], \
            filedata['valid_x'], filedata['valid_y'], \
            filedata['test_x'], filedata['test_y']

In [ ]:
genre_model8 = keras.models.load_model('The Movies Dataset/genre8_0.5718.hdf5')

In [7]:
filepath = './The Movies Dataset/genre_dataset.npz'
genre_uid2idx, genre_iid2idx, _, _, _, _, test_x, test_y = load_dataset(filepath)

In [8]:
genre_model8.evaluate(
    x=(test_x[:,0,None], test_x[:,1,None]),
    y=test_y
)

625/625 [==============================] - 0s 446us/step - loss: 0.5676 - binary_accuracy: 0.6946


[0.5676388740539551, 0.6946499943733215]

In [118]:
filepath = './The Movies Dataset/rcmd_dataset.npz'
rcmd_uid2idx, rcmd_iid2idx, _, _, _, _, test_x, test_y = load_dataset(filepath)

In [119]:
rcmd_model24 = keras.models.load_model('The Movies Dataset/model24_0.4762.hdf5')

In [120]:
rcmd_model24.evaluate(
    x=(test_x[:,0,None], test_x[:,1,None]),
    y=test_y
)

625/625 [==============================] - 0s 441us/step - loss: 0.4805 - binary_accuracy: 0.7704


[0.48053163290023804, 0.7703999876976013]

불러온 벡터들을 이어붙여 모델의 학습 데이터를 만듭니다.

장르게 여러 개인 영화의 경우 장르 벡터들의 평균을 사용합니다.

In [16]:
get_genre_uidx = genre_uid2idx.get
get_genre_iidx = dict(genre_df.values[::, ::-1]).get
get_rcmd_uidx = rcmd_uid2idx.get
get_rcmd_iidx = rcmd_iid2idx.get


def voctor_getter(model, layer, get_idx):
    layer_num = {'user': 2, 'item': 3}[layer]
    vectors = model.layers[layer_num].get_weights()[0]
    def func(_id):
        idx = get_idx(_id)
        return None if idx is None else vectors[idx]
    return func


get_genre_user_vec = voctor_getter(genre_model8, 'user', get_genre_uidx)
get_genre_item_vec = voctor_getter(genre_model8, 'item', get_genre_iidx)
get_rcmd_user_vec = voctor_getter(rcmd_model24, 'user', get_rcmd_uidx)
get_rcmd_item_vec = voctor_getter(rcmd_model24, 'item', get_rcmd_iidx)


def get_user_vec(user_id):
    return np.concatenate((get_genre_user_vec(user_id), get_rcmd_user_vec(user_id)))

def get_item_vec(genre_names, imdb_id):
    genre_vectors = np.array([get_genre_item_vec(genre) for genre in genre_names])
    return np.concatenate((genre_vectors.mean(axis=0), get_rcmd_item_vec(imdb_id)))

In [17]:
data_x = list()
data_y = genre_rating_df['prefer'].values
for _, row in tqdm(genre_rating_df.iterrows()):
    user_vec = get_user_vec(row['userId'])
    item_vec = get_item_vec(row['genres'].split(', '), row['imdbId'])
    data_x.append(np.concatenate((user_vec, item_vec)))

data_x = np.array(data_x)
data_x.shape, data_y.shape

0it [00:00, ?it/s]

((25946710, 64), (25946710,))

In [20]:
train_x, valid_x, train_y, valid_y = train_test_split(
    data_x, data_y, test_size=40000,
)
valid_x, test_x, valid_y, test_y = train_test_split(
    valid_x, valid_y, test_size=20000,
)
np.savez_compressed(
    './The Movies Dataset/genre_rcmd_vector_dataset.npz',
    **{
        'train_x': train_x, 
        'train_y': train_y, 
        'valid_x': valid_x, 
        'valid_y': valid_y, 
        'test_x': test_x, 
        'test_y': test_y,
    },
)

In [122]:
with np.load('./The Movies Dataset/genre_rcmd_vector_dataset.npz') as filedata:
    train_x = filedata['train_x']
    train_y = filedata['train_y']
    valid_x = filedata['valid_x']
    valid_y = filedata['valid_y']
    test_x = filedata['test_x']
    test_y = filedata['test_y']

# SVM model

SVM classifier를 이용해 prefer를 예측합니다.

SVM은 학습 데이터가 2천 만 개나 필요할 정도로 강력한 모델이 아니므로 준비된 학습 데이터 중 10만 개만 사용합니다.

테스트 데이터셋에서 정확도 79.73%로 기존보다 약 2.7%p 개선된 결과를 얻을 수 있었습니다.

In [114]:
def evaluate(model, x, y):
    pred = model.predict(x)
    pred_ratio = pred.sum() / len(y)
    y_ratio = y.sum() / len(y)
    acc = (y == pred).sum() / len(y)
    print(f'{pred_ratio=}, {y_ratio=}, {acc=}')

In [115]:
%%time
sample_size = 100000
rbf_svc = svm.SVC()
rbf_svc = rbf_svc.fit(train_x[:sample_size], train_y[:sample_size])

pred_ratio=0.68045, y_ratio=0.6184, acc=0.79385
CPU times: user 19min 40s, sys: 156 ms, total: 19min 40s
Wall time: 19min 41s


In [123]:
evaluate(rbf_svc, valid_x, valid_y)

pred_ratio=0.68045, y_ratio=0.6184, acc=0.79385


In [63]:
evaluate(rbf_svc, test_x, test_y)

pred_ratio=0.6868, y_ratio=0.6216, acc=0.7973


# Decision Tree Model

Decision tree를 이용해 prefer를 예측합니다.

이번에는 72.87%로 기존보다 낮은 정확도가 나왔습니다.

In [69]:
sample_size = 100000
tree_clf = tree.DecisionTreeClassifier(max_depth=5)
tree_clf = tree_clf.fit(train_x[:sample_size], train_y[:sample_size])

In [70]:
evaluate(tree_clf, valid_x, valid_y)

pred_ratio=0.69205, y_ratio=0.6184, acc=0.72555


In [71]:
evaluate(tree_clf, test_x, test_y)

pred_ratio=0.6935, y_ratio=0.6216, acc=0.7287


# Logistic Regression

Logistic regression으로 prefer를 예측합니다.

이번에는 validation으로 시도해 본 결과, 20만 개 데이터를 학습했을 때, 10만 개 데이터를 학습했을 때보다 정확도가 높아 20만 개 데이터를 사용했습니다.

그 이후로는 눈에 띄는 정확도 향상은 없었습니다.

결과는 74.95%로 기존 모델보다 낮았습니다.

In [110]:
sample_size = 200000
lin_clf = linear_model.LogisticRegression(
    max_iter=500,
)
lin_clf = lin_clf.fit(train_x[:sample_size], train_y[:sample_size])

In [111]:
evaluate(lin_clf, valid_x, valid_y)

pred_ratio=0.6818, y_ratio=0.6184, acc=0.7414


In [112]:
evaluate(lin_clf, test_x, test_y)

pred_ratio=0.6812, y_ratio=0.6216, acc=0.7495
